In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import fl_utils.fl_interpretable_functions as interpret
import fl_utils.fl_riskslim as slim
import fl_utils.fl_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\fl-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [3]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_six_month'].values

In [4]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
np.mean(cart_summary['KY_score']), np.mean(ebm_summary['KY_score'])

(0.579305601562677, 0.6070498654599928)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_stumps").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_stumps").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_six_month'].values

## columns 
cols = KY_X.columns[1:]

In [7]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.08, 
                                        cols, 
                                        816)
single_stump_model['features']

['sex1',
 'age_at_current_charge21',
 'age_at_current_charge27',
 'age_at_current_charge36',
 'age_at_current_charge51',
 'p_arrest4',
 'p_arrest6',
 'p_arrest8',
 'p_charges1',
 'p_charges2',
 'p_violence1',
 'p_misdemeanor1',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_property2',
 'p_property3',
 'p_weapon1',
 'p_felassult1',
 'p_misdeassult1',
 'p_drug2',
 'p_drug3',
 'p_dui1',
 'p_fta_two_year1',
 'p_pending_charge5',
 'p_probation2',
 'p_probation3',
 'six_month1',
 'one_year1']

In [8]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary['best_params'], np.mean(stumps_summary['KY_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.03}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}],
 0.6127768443903779,
 0.11510713806119817)

### RiskSLIM

In [10]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.03,   
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

11

In [11]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [12]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'drug_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
12/27/19 @ 12:57 PM | 163 rows in lookup table
12/27/19 @ 12:57 PM | ------------------------------------------------------------
12/27/19 @ 12:57 PM | runnning initialization procedure
12/27/19 @ 12:57 PM | ------------------------------------------------------------
12/27/19 @ 12:57 PM | CPA produced 2 cuts
12/27/19 @ 12:57 PM | running naive rounding on 21 solutions
12/27/19 @ 12:57 PM | best objective value: 0.1477
12/27/19 @ 12:57 PM | rounding produced 5 integer solutions
12/27/19 @ 12:57 PM | best objective value is 0.1506
12/27/19 @ 12:57 PM | running sequential rounding on 21 solutions
12/27/19 @ 12:57 PM | best objective value: 0.1477
12/27/19 @ 12:57 PM | sequential rounding produced 4 integer solutions
12/27/19 @ 12:57 PM | best objective value: 0.1496
12/27/19 @ 12:57 PM | polishing 9 solutions
12/27/19 @ 12:57 PM | best objective value: 0.1496
12/27/19 @ 12:57 PM | polishing produced 4 integer solutions
12/27/19 @

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:57 PM | adding 57 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1496.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:57 PM | adding 53 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1490.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:57 PM | adding 56 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1415.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:57 PM | adding 54 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1418.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:58 PM | adding 53 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1532.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:58 PM | adding 57 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1526.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:58 PM | adding 53 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1502.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:58 PM | adding 61 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1690.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 12:58 PM | adding 57 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1574.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable

In [14]:
np.mean(riskslim_summary['FL_validation']), np.mean(riskslim_summary['KY_score'])

(0.6801903505516667, 0.6369117140565704)

In [15]:
#### save results
summary_drug6_fl_inter_model = {"cart": cart_summary,
                                "ebm": ebm_summary,
                                "stumps": stumps_summary,
                                "riskslim": riskslim_summary}

In [17]:
path = "./results/interpretable/six-month/"
results = [["Drug",             
            np.str(round(np.mean(cart_summary['KY_score']),3)) + " (" + np.str(round(np.std(cart_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['KY_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['KY_score']),3)) + " (" + np.str(round(np.std(stumps_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['KY_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['KY_score']),3)) + ")"]]
with open(path + 'six-month-fl-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)